# Theoretical background
In classical mechanics, Noether’s theorem tells us that symmetries imply conservation laws. For example, if a Lagrangian is invariant under rotation, then angular momentum is conserved. In quantum mechanics, we are also interested in studying how certain features of a system are affected by physical transformations. The first transformation that we usually encounter when learning quantum mechanics is time evolution, where we ask, “what happens to a quantum state as time goes forward?” Time evolution is one example of what we generally call a **symmetry transformation**: for any two states $|x\rangle$ and $|y\rangle$, the transition probability between them is unaffected by the transformation, i.e.,

$$\left|\left\langle x\middle|\ y\right\rangle\right|=\left|\left\langle\widetilde{x}\middle|\widetilde{y}\right\rangle\right|.$$

Furthermore, **Wigner’s theorem** tells us that all symmetry transformations are either *unitary*, $$\left\langle\widetilde{x}\middle|\widetilde{y}\right\rangle=\left\langle x\middle| y\right\rangle,$$ or *anti-unitary*, $$\left\langle\widetilde{x}\middle|\widetilde{y}\right\rangle=\left\langle y\middle| x\right\rangle.$$ 

Next, we consider a specific symmetry transformation known as *time reversal*. As the name suggests, time reversal affects time evolution by reversing the direction of time, which changes quantities like momentum but leaves quantities like position unchanged (for a more rigorous explanation, see a standard quantum mechanics book like [Sakurai](https://en.wikipedia.org/wiki/Modern_Quantum_Mechanics)). Symmetry transformations also preserve the canonical commutation relations for position/momentum, $$\left[\hat{x},\hat{p}\right]=i\hbar,$$ and spin, $$\left[{\hat{S}}_i,{\hat{S}}_j\right]=i{\hbar\epsilon}_{ijk}{\hat{S}}_k.$$

Combining these ideas allows us to draw two fundamental conclusions (we will skip the detailed derivation for now).
1. The time reversal transformation is anti-unitary.
2. If a system of spin-1/2 particles (like electrons) is invariant under time reversal (formally, $\left[\hat{H},\hat{T}\right]=0$ where $\hat{H}$ is the Hamiltonian and $\hat{T}$ is the time reversal operator), then every energy eigenstate $|\psi\rangle$ transforms in such a way that $|\widetilde{\psi}\rangle$ is a different state with the same energy.

The second conclusion, called the **Kramers degeneracy theorem**, immediately implies that all energy levels of the system have a degeneracy of (at least) two. The two states $|\psi\rangle$ and $|\widetilde{\psi}\rangle$ are known as a **Kramers pair**, which will become relevant shortly.

We now briefly turn our attention to topology. For a family of 0D systems described generically by an eigenvalue equation
$$\hat{H}|n\rangle=E_n|n\rangle,$$
two systems are topologically equivalent if and only if they have the same number of filled energy levels. Otherwise, a continuous and gap-preserving transformation between the two Hamiltonians does not exist.  For materials, this analysis becomes more complicated because there is an additional quantum number $k$ that arises from the discrete translational symmetry of the system:
$$\hat{H}|n,k\rangle=E_n(k)|n,k\rangle$$
As before, our goal is to deduce whether two Hamiltonians are topologically equivalent based on their energy spectra. But in this case, the spectrum $E_n(k)$ is just the band structure, which we already know how to calculate with DFT. 

So then, given the band structures of two different materials, how can we determine whether they have the same topology? That question has driven extensive research in physics and materials science since the early 2000s. For certain types of materials, we now have a much better understanding of the mechanism that dictates topological classification. Bi$_2$Se$_3$ is one of those materials. Because it is an insulator with time reversal symmetry, it turns out that the topology can be characterized by studying its Kramers pairs. The algorithm that we will use was first developed by Soluyanov and Vanderbilt in 2011 (PRB 83, 235401).  It relies on the properties of Wannier functions, which will not be explained here for the sake of brevity. If you are interested, you are encouraged to read the above reference as well as PRB 95, 075146 (2017).

# Overview of the calculation
- Use the crystal structure to calculate the DFT band structure of Bi$_2$Se$_3$ in VASP. As in previous labs, this requires two steps: 
    - a self-consistent calculation sampling the full Brillouin zone, and
    - a second calculation with fixed charge density that evaluates the spectrum along high-symmetry lines. 

- Generate a model Hamiltonian using the tight binding method:
    - Use VASP to create the input files (overlap matrices and orbital projections) needed for Wannier90.
    - Use Wannier90 to write the Hamiltonian in the Wannier orbital basis.
- Calculate the topological invariant using two methods:
    - Manual
        - Load the model Hamiltonian from the previous step into Python using the tbmodels package.
        - Apply the Soluyanov-Vanderbilt algorithm by using Z2Pack to calculate hybrid Wannier charge centers and identifying the Kramers pairs. 
    - Automatic
        - Set up a series of VASP + Wannier90 calculations using Z2Pack, bypassing the tight binding step. 
        - **Note: in VASP, this currently only works for insulators.** For other materials (e.g., topological semimetals), the manual technique is required. Alternatively, another first-principles code, like ABINIT, can be used instead of VASP. 


### Part 1: SCF calculation
1. There are 6 subfolders according to the steps outlined above. First, `cd 1_scf`.
2. Browse each of the input files provided. At this point, you should be familiar with what each file represents. In the `INCAR`, notice that we are including spin-orbit coupling in the calculation. In principle, we could repeat all calculations without SOC, but this step is omitted from the tutorial in the interest of time. If we did not include SOC, we would draw the incorrect conclusion that Bi$_2$Se$_3$ is topologically trivial, which gives us some insight into the underlying physical mechanism governing its topological properties.
3. Generate the `POTCAR` file with `TITEL` tags `Bi` and `Se`. 
4. Submit the job. Convergence will take about 17 minutes. 
5. Open the `EIGENVAL` file. The first six lines contain general information about the calculation. The remaining lines contain the full dataset for $E_n(k)$. Each $k$ point in the grid (corresponding to the `KPOINTS` file) has its own block, and each block begins with the coordinates of the point, written in the basis of the primitive reciprocal lattice vectors. For example, line 8 begins the block at the $\Gamma$ point $(0.0, 0.0, 0.0)$. Each of the remaining lines displays the band index, energy, and occupation number in that order. 
6. What pattern do you notice in the energy spectrum? What causes the pattern to emerge? (Hint: consider the symmetries of the material)

### Part 2: band structure
1. `cd ../2_bands` and copy the `INCAR`, `POSCAR`, `POTCAR`, `CHGCAR`, and `VASP.bash` from `../1_scf`. 
2. Make the following changes to the INCAR:
```
ICHARG = 11
ISMEAR = 0
LCHARG = False
LWAVE = False
NELMIN = 15
```

3.	Submit the job. Convergence will take about 3 minutes. In the meantime, if you have not already done so, install the plotting software sumo using `pip install --user sumo`.
4. Plot the band structure using `sumo-bandplot --format png`. Do not skip this step! The sumo output will be required in Part 4. 


### Part 3: Wannier90 setup
1. cd `../3_w90_setup`. For this part, the goal is to use the Bloch states (from the `WAVECAR`) calculated in Part 1 to calculate two sets of matrices: (i) the overlap matrices (`wannier90.mmn`), determined by taking specific inner products between Bloch states, and (ii) the projection matrices (`wannier90.amn`), determined by projecting each Bloch state onto trial orbitals. To accomplish this, we use a modified version of VASP that was written to be compatible with Wannier90 (in VASP, this is known as the W90 “library mode”). We first read the `WAVECAR` using the `ISTART` tag, then pretend to do a DFT calculation while not actually taking any electronic steps (this is done by setting `NELM` to 0). We also include some tags that instruct VASP on which Bloch states and trial orbitals to consider in the calculation. 
2. Copy the `INCAR`, `KPOINTS`, `POSCAR`, `POTCAR`, and `VASP.bash` from `../../1_scf`. Then, using `mv` rather than `cp`, move the `WAVECAR` from `../../1_scf` to your folder. This is done to prevent unnecessary duplication of large files. 
3. Open the `INCAR` and make the changes described above:
```
ISTART = 1
NELM = 0
LWAVE = False
LCHARG = False
ISMEAR = 0
LWANNIER90 = True
NUM_WANN = 30
WANNIER90_WIN = "
exclude_bands : 1-10
Begin Projections
Bi:l=1
Se:l=1
End Projections
"
```

Note that the `WANNIER90_WIN` tag spans multiple lines and includes quotation marks. 
4. Submit the job. There is no convergence to keep track of because the calculation only consists of basic linear algebra operations according to the definitions of the overlap and projection matrices. This will take a while, about 50 minutes.

### Part 4: Wannier90 tight binding model
1. Now that you have the inputs needed for Wannier90, you can finally construct a tight binding Hamiltonian. Enter `4_w90_run`. Copy the `wannier90.win`, `wannier90.mmn`, `wannier90.amn`, and `wannier90.eig` files from `../3_w90_setup`.
2. Open `wannier90.win`, which is the instruction file for Wannier90 (similar to the `INCAR` for VASP). You will need to add several lines. First, some basic tags:
```
guiding_centres = .true.
num_iter = 0
write_hr = .true.
write_xyz = .true.
bands_plot = .true.
```
3. Next, in order to verify that our final tight binding model is accurate, we will compare its energy spectrum to the DFT band structure. Therefore, Wannier90 needs to know which high symmetry lines to evaluate the Hamiltonian along. Add these lines:
```
begin kpoint_path
G 0.000   0.000   0.000   L 0.500   0.000   0.000
L 0.500   0.000   0.000   Z 0.500   0.500   0.500
Z 0.500   0.500   0.500   F 0.500   0.500   0.000
F 0.500   0.500   0.000   G 0.000   0.000   0.000
G 0.000   0.000   0.000   Z 0.500   0.500   0.500
end kpoint_path
```
4. Finally, we need to specify tags related to the disentanglement procedure. Basically, when we construct the tight binding Hamiltonian, we only consider a subset of the original DFT band structure. In doing so, we run into issues related to the fact that bands higher in energy are not isolated from one another. Disentanglement ensures that the resulting energy spectrum is smooth and matches the DFT band structure as closely as possible for higher bands. For more a more detailed explanation, see PRB 56, 12847 (1997). Here are the settings:
```
dis_num_iter = 1000
dis_conv_tol = 1.0E-11
dis_win_min = -2.92
dis_win_max = 12.58
dis_froz_min = -2.92
dis_froz_max = 6.58
```
5. Close the `wannier90.win` file and submit the job. It should converge in about 8 minutes.
6. Now, compare the two energy spectra using the plotting script. As intended, disentanglement has caused the Wannier bands above 3 eV to remain smooth while closely aligning with the DFT bands. 

### Part 5: Wannier90 tight binding model
1. We are finally ready to calculate the topological invariant. Enter `5_hwcc_model`. We will use Z2Pack to calculate the hybrid Wannier charge centers on specific surfaces within the first Brillouin zone. For a detailed explanation of how this works, see PRB 95, 075146 (2017). The script `hwcc.py` has already been provided for you, so all you need to do is submit the job. It will take about 5 minutes.
2. Open the `invariant` file. For materials in the symmetry class of Bi$_2$Se$_3$, there are two unique topological invariants. The strong invariant is given in line 1, while the weak invariant* is given in line 2. This result tells us that Bi$_2$Se$_3$ is a strong topological insulator. We will also verify it ourselves. **There are technically three weak invariants, but for Bi$_2$Se$_3$, symmetry under exchange of crystal axes forces them to all be equal.*
3. Plot the full HWCC data using the included plotting script. There should be two figures, each corresponding to a different surface within the Brillouin zone. One is at $k_x = 0$ and the other is at $k_x = 0.5$. Using these plots, we will apply the Soluyanov-Vanderbilt algorithm to verify the Z2Pack result.

Here, the HWCCs also come in Kramers pairs. Topological invariants are determined by tracking the evolution of each pair as the $k$ parameter varies from 0 to 1. In this case, degeneracy is only guaranteed at the endpoints. The idea is to check whether the states at $k = 0$ have “switched partners” compared to those at $k = 1$. The algorithm is as follows: for a given surface, draw a horizontal reference line. If the HWCCs cross the line an odd (even) number of times, then the Z2 invariant on that surface is 1 (0). For $k_x = 0$, the line is crossed once near $k = 0.05$. For $k_x = 0.5$, the line is never crossed. 

### Part 6: Automatic HWCC calculation
1. It is actually possible to do the same HWCC calculation without needing to construct a tight binding model, by automating VASP with Z2Pack (again, this is only possible for insulators in the current version of VASP). Enter `6_hwcc_auto`. In the input folder, we will prepare VASP input files as though we were doing a band structure calculation, but with a few changes. First, copy the `INCAR`, `POSCAR`, `POTCAR`, and `CHGCAR` from `2_bands`. Notice that the `KPOINTS` file is not copied. Z2Pack will prepare a series of VASP calculations, each using a different `KPOINTS` file according to the surface being studied in the Brillouin zone.
2. Open the INCAR and make the following changes:
```
ISYM = -1
LWANNIER90 = True
NUM_WANN = 18
WANNIER90_WIN = "
num_iter 0
exclude_bands : 1-10, 29-42
shell_list 1
skip_b1_tests = .true.
"
```
3. In the manual calculation, we used a very dense grid (100 $k$ points) so that the Soluyanov-Vanderbilt algorithm could easily be visualized. In principle, it is not necessary. Z2Pack uses extremely strict convergence criteria when determining the final topological invariant and will add more points if necessary. As long as the calculation converges (which can be checked in the output file), we can usually be quite confident in the result. Hence, we are now using the default value of 10 initial k points. These parameters are controlled in the `hwcc.py` script. You can take a closer look at the details if you are interested in studying topological materials in the future. For now, submit the job, and it should converge in about 16 minutes.
4. Run the plotting script. To illustrate the method Z2Pack uses “under the hood” for calculating topological invariants, blue diamonds are included in the plot. They are centered inside the largest gap between adjacent HWCCs. After finishing the calculation, Z2Pack checks the number of times the diamonds “jump over” the gray circles. 

If you made it this far, **congratulations**!! You now know how to use DFT to study the topological properties of insulators. Of course, this is not the be-all and end-all. Some key concepts are missing from this kind of study. What if the material is not an insulator? How accurate is this method, considering that we use the band structure from DFT? To explore these more advanced questions, continue to other sections of `Topological`.